### recount quick start

Follows this guide: http://leekgroup.github.io/recount/recount-quickstart.html

In [1]:
## Load libraries
library('recount')
library('SummarizedExperiment')
library('SciServer')

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, cbind, colMeans, colSums, colnames, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, lengths, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, rank, rbind, rowMeans, rowSums, rownames, sapply,
    setdiff, sort, table, tapply, union, unique, unsp

In [2]:
## Locally-mounted directory where recount resource resides
recount_dir <- '/home/idies/workspace/recount01'

In [3]:
## Find a project of interest
project_info <- abstract_search('GSE32465')
project_info

,number_samples,species,abstract,project
340,12,human,"Summary: K562-shX cells are made in an effort to validate TFBS data and ChIP-seq antibodies in Myers lab (GSE32465). K562 cells are transduced with lentiviral vector having Tet-inducible shRNA targeting a transcription factor gene. Cells with stable integration of shRNA constructs are selected using puromycin in growth media. Doxycycline is added to the growth media to induce the expression of shRNA and a red fluorescent protein marker. A successful shRNA cell line shows at least a 70% reduction in expression of the target transcription factor as measured by qPCR. For identification, we designated these cell lines as K562-shX, where X is the transcription factor targeted by shRNA and K562 denotes the parent cell line. For example, K562-shATF3 cells are K562 derived cells selected for stable integration of shRNA targeting the transcription factor ATF3 gene and showed at least a 70% reduction in the expression of ATF3 gene when measured by qPCR. Cells growing without doxycycline (uninduced) are used as a control to measure the change in expression of target transcription factor gene after induction of shRNA using doxycycline. For detailed growth and culturing protocols for these cells please refer to http://hudsonalpha.org/myers-lab/protocols . To identify the potential downstream targets of the candidate transcription factor, analyze the mRNA expression profile of the uninduced and induced K562-shX using RNA-seq. For data usage terms and conditions, please refer to http://www.genome.gov/27528022 and http://www.genome.gov/Pages/Research/ENCODE/ENCODEDataReleasePolicyFinal2008.pdf Overall Design: Make K562-shX cells as described in the http://hudsonalpha.org/myers-lab/protocols . Measure the mRNA expression levels in uninduced K562-shX and induced K562-shX cells in two biological replicates using RNA-seq. Identify the potential downstream targets of the candidate transcription factor.",SRP009615


In [4]:
study <- project_info$project  # SRP009615
study_dir <- file.path(recount_dir, study)

In [5]:
## Load the data

In [6]:
load(file.path(study_dir, 'rse_gene.Rdata'))

In [7]:
## Find the GEO accession ids
# (NCBI connection closed can happen error here sometimes)
geoids <- sapply(colData(rse_gene)$run, find_geo)

In [8]:
username = Authentication.getKeystoneUserWithToken(Authentication.getToken())$userName

In [9]:
## Get the sammple information from GEO
geoinfo <- lapply(geoids, function(x) {
    geo_info(x, destdir=paste0('/home/idies/workspace/Temporary/', username, '/scratch')) })

Using locally cached version of GSM836270 found here:
/home/idies/workspace/Temporary/jkim485/scratch/GSM836270.soft 
Using locally cached version of GSM836271 found here:
/home/idies/workspace/Temporary/jkim485/scratch/GSM836271.soft 
Using locally cached version of GSM836272 found here:
/home/idies/workspace/Temporary/jkim485/scratch/GSM836272.soft 
Using locally cached version of GSM836273 found here:
/home/idies/workspace/Temporary/jkim485/scratch/GSM836273.soft 
Using locally cached version of GSM847561 found here:
/home/idies/workspace/Temporary/jkim485/scratch/GSM847561.soft 
Warning message in `[<-`(`*tmp*`, i, value = IRanges::CharacterList(unlist(unname(result[i])))):
"implicit list embedding of S4 objects is deprecated"Using locally cached version of GSM847562 found here:
/home/idies/workspace/Temporary/jkim485/scratch/GSM847562.soft 
Warning message in `[<-`(`*tmp*`, i, value = IRanges::CharacterList(unlist(unname(result[i])))):
"implicit list embedding of S4 objects is dep

In [10]:
## Extract the sample characteristics
geochar <- lapply(geoinfo, geo_characteristics)

In [11]:
## Note that the information for this study is a little inconsistent, so we
## have to fix it.
geochar <- do.call(rbind, lapply(geochar, function(x) {
    if('cells' %in% colnames(x)) {
        colnames(x)[colnames(x) == 'cells'] <- 'cell.line'
        return(x)
    } else {
        return(x)
    }
}))

In [12]:
## We can now define some sample information to use
sample_info <- data.frame(
    run = colData(rse_gene)$run,
    group = sapply(geoinfo, function(x) { ifelse(grepl('uninduced', x$title),
        'uninduced', 'induced') }),
    gene_target = sapply(geoinfo, function(x) { strsplit(strsplit(x$title,
        'targeting ')[[1]][2], ' gene')[[1]][1] })
)

In [13]:
## Scale counts by taking into account the total coverage per sample
rse <- scale_counts(rse_gene)

In [14]:
## Add sample information for DE analysis
colData(rse)$group <- sample_info$group
colData(rse)$gene_target <- sample_info$gene_target

In [15]:
## Perform differential gene expression analysis with DESeq2
library('DESeq2')

## Specify design and switch to DESeq2 format
dds <- DESeqDataSet(rse, ~ gene_target + group)

# Workaround for: https://support.bioconductor.org/p/111792/
mcols(dds)$symbol <- unlist(lapply(mcols(dds)$symbol, paste, collapse=","))

## Perform DE analysis
dds <- DESeq(dds, test = 'LRT', reduced = ~ gene_target, fitType = 'local', parallel=T)
res <- results(dds)

converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates: 14 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 14 workers


In [16]:
res

log2 fold change (MLE): group uninduced vs induced 
LRT p-value: '~ gene_target + group' vs '~ gene_target' 
DataFrame with 58037 rows and 6 columns
                      baseMean log2FoldChange     lfcSE      stat     pvalue
                     <numeric>      <numeric> <numeric> <numeric>  <numeric>
ENSG00000000003.14   12.014354      0.7562448 0.5427437  1.797677 0.17999355
ENSG00000000005.5     2.577085      1.3085612 1.0041180  1.253870 0.26281455
ENSG00000000419.12 2010.759283     -0.3427947 0.1365863  6.270242 0.01227824
ENSG00000000457.13  744.840210     -0.2424541 0.1271758  3.628092 0.05681192
ENSG00000000460.16 1358.236026     -0.3719585 0.1410730  6.926779 0.00849145
...                        ...            ...       ...       ...        ...
ENSG00000283695.1    0.2624722     -1.0197790 3.1039716 0.1779536  0.6731378
ENSG00000283696.1   15.0206855     -0.1778908 0.4343602 0.1628430  0.6865524
ENSG00000283697.1   30.7821042      0.2595221 0.2837579 0.8294314  0.3624370
ENSG

In [ ]:
## Explore gene-level results
plotMA(res, main="DESeq2 results for SRP009615")